# Train and Test custom YOLOv5 on Amazon SageMaker Studio

In this notebook we will train and test a custom YOLOv5 object detection CV model within Amazon SageMaker Studio. 

**Steps:**

0. Initial configuration.
1. Download a labeled dataset.
3. Train the custom YOLOv5 model.
4. Make predictions against the created model. 


| ⚠️ WARNING: For this notebook to work, make sure to select the following settings in your jupyter environment: |
| -- |
Image: "PyTorch 1.10 Python 3.8 GPU Optimized"
Instance_type: "ml.g4dn.xlarge" (fast launch)

## 0. Initial Configuration

#### Download the YOLOv5 repository

In [ ]:
!git clone --quiet https://github.com/ultralytics/yolov5
!pip install -qr yolov5/requirements.txt

In [ ]:
import os
import boto3
import glob
s3_resource = boto3.resource('s3')

## 1. Download a labeled dataset with YOLOv5 expected format.

Before we train a custom YOLOv5 model, we need to have a labeled dataset. 
In the previous notebook "0 - Label your dataset with Amazon SageMaker GroundTruth" you will be able to label your own dataset and transform it into YOLOv5 expected format or use an example custom dataset. Once you have run through one of the two options you will have available the S3 dataset location and labels used.

In [ ]:
dataset_s3_uri = ""
labels = []

#### Download the dataset

In [ ]:
def split_s3_path(s3_path):
    path_parts=s3_path.replace("s3://","").split("/")
    bucket=path_parts.pop(0)
    key="/".join(path_parts)
    return bucket, key

In [ ]:
bucket,dataset_name = split_s3_path(dataset_s3_uri)
bucket,dataset_name

In [ ]:
def download_dataset(bucket_name, folder):
    bucket = s3_resource.Bucket(bucket_name) 
    for obj in bucket.objects.filter(Prefix = folder):
        if not os.path.exists(os.path.dirname(obj.key)):
            os.makedirs(os.path.dirname(obj.key))
        bucket.download_file(obj.key, obj.key)

In [ ]:
download_dataset(bucket, dataset_name)

#### Lets explore our dataset

In [ ]:
for filename in glob.iglob(dataset_name + '**/**', recursive=True):
     print(filename)

#### Now let's add these data sources to the data library in the yolov5 folder for our model to train

In [ ]:
with open("yolov5/data/custom-model.yaml", 'w') as target:
    target.write("path: ../{}\n".format(dataset_name))
    target.write("train: images/train\n")
    target.write("val: images/validation\n")
    target.write("names:\n")
    for i, label in enumerate(labels):
        target.write("  {}: {}\n".format(i, label))
        
with open('yolov5/data/custom-model.yaml') as file:
    lines = file.readlines()
    for line in lines:
        print(line)

## 3. Train the custom YOLOv5 model.

In [ ]:
!python yolov5/train.py --workers 4 --device 0 --img 640 --batch 8 --epochs 10 --data yolov5/data/custom-model.yaml --weights yolov5s.pt --cache

## 4. Make inferences with the created model.

In [ ]:
!python yolov5/detect.py --weights yolov5/runs/train/exp/weights/best.pt --img 640 --conf 0.5 --source ""

| ⚠️ WARNING: Remember to shutdown the instance once finalized with this notebook to prevent unnecesary charges. Head to running Terminals and Kernels tab and shutdown the running instance. |
| -- |